<a href="https://colab.research.google.com/github/stazam/Datamining2-project/blob/main/model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [14]:
from google.colab import files 

uploaded = files.upload()

Saving sciencetech_articles.npy to sciencetech_articles (1).npy
Saving sport_articles.npy to sport_articles (1).npy
Saving travel_articles.npy to travel_articles (1).npy


In [15]:
import numpy as np 

science = np.load('sciencetech_articles.npy',allow_pickle='TRUE').item()
sport = np.load('sport_articles.npy',allow_pickle='TRUE').item()
travel = np.load('travel_articles.npy',allow_pickle='TRUE').item()

odstanit prazdne stringy a url adresy spojit do jedneho stringu 



In [16]:
def preprocess_articles(articles) -> list:

  articles_preprocess = []

  for key,value in articles.items():
    list_temp = []
    list_temp.append(key)
    for text in value:
      if len(text) > 0:
        list_temp.append(text.lower().strip().rstrip())
    articles_preprocess.append(list_temp)

  articles_preprocess = [' '.join(item) for item in articles_preprocess]

  return articles_preprocess 

In [17]:
sport = preprocess_articles(sport)
labels_sport = [0] * len(sport)

travel = preprocess_articles(travel)
labels_travel = [1] * len(travel)

science = preprocess_articles(science)
labels_science = [2] * len(science)

articles = sport + travel + science
labels =  labels_sport + labels_travel + labels_science

In [25]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(articles, labels, test_size=0.33, random_state=42)


In [19]:
import tensorflow as tf

from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [20]:
max_length = 10000
trunc_type='post'
padding_type='post'
oov_tok = "<OOV>"
training_size = 20000
vocab_size = 10700

In [21]:
tokenizer = Tokenizer(num_words=vocab_size,oov_token=oov_tok)
tokenizer.fit_on_texts(X_train)

word_index = tokenizer.word_index

X_train = tokenizer.texts_to_sequences(X_train)
X_train_padded = np.array(pad_sequences(X_train, maxlen=max_length, padding=padding_type, truncating=trunc_type))

X_test = tokenizer.texts_to_sequences(X_test)
X_test_padded = np.array(pad_sequences(X_test, maxlen=max_length, padding=padding_type, truncating=trunc_type))

In [22]:
y_train = tf.keras.utils.to_categorical(y_train, num_classes=3)
y_test = tf.keras.utils.to_categorical(y_test, num_classes=3)

In [23]:
import keras
from keras import applications
from keras.models import Sequential
from keras.layers import Dense, Embedding, Bidirectional
from keras.layers import Convolution2D, MaxPooling2D,BatchNormalization,GlobalAveragePooling1D, Flatten, Dropout
from keras import optimizers

In [24]:
model = Sequential([
    Embedding(vocab_size, 512, input_length=max_length),
    Bidirectional(keras.layers.LSTM(150, return_sequences=True)),
    Bidirectional(keras.layers.LSTM(64, return_sequences=True)),
    Bidirectional(keras.layers.LSTM(32, return_sequences=True)),
    Dropout(0.5),
    Flatten(),
    Dense(128, activation='relu'),
    Dense(128, activation='relu'),
    Dense(64, activation='relu'),
    Dense(64, activation='relu'),
    Dense(24, activation='relu'),
    Dropout(0.5),
    Dense(3, activation='softmax')
])
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

num_epochs = 30
history = model.fit(X_train_padded, y_train, epochs=num_epochs, validation_data=(X_test_padded, y_test))

Epoch 1/30
3/3 [==============================] - 38s 11s/step - loss: 3.5527 - accuracy: 0.3659 - val_loss: 3.5331 - val_accuracy: 0.2927
Epoch 2/30
3/3 [==============================] - 25s 9s/step - loss: 4.3384 - accuracy: 0.3415 - val_loss: 8.3381 - val_accuracy: 0.3415
Epoch 3/30
3/3 [==============================] - 25s 9s/step - loss: 6.2782 - accuracy: 0.3415 - val_loss: 3.1053 - val_accuracy: 0.2927
Epoch 4/30


KeyboardInterrupt: ignored

In [ ]:
import matplotlib.pyplot as plt


def plot_graphs(history, string):
  plt.plot(history.history[string])
  plt.plot(history.history['val_'+string])
  plt.xlabel("Epochs")
  plt.ylabel(string)
  plt.legend([string, 'val_'+string])
  plt.show()
  
plot_graphs(history, "accuracy")
plot_graphs(history, "loss")

In [ ]:
sequences = tokenizer.texts_to_sequences(sentence)
padded = pad_sequences(sequences, maxlen=max_length, padding=padding_type, truncating=trunc_type)
print(model.predict(padded))